# アドバンスステップ第1回課題
小売店の売上予測

In [59]:
#ライブラリインポート
%matplotlib inline
import pandas as pd
import pandas.io.sql as psql
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import scipy as sp
from scipy import stats
from scipy.stats import spearmanr

import psycopg2
from pandas import DataFrame
from __future__ import print_function

#xgboostをインポート
import xgboost as xgb
from sklearn.cross_validation import StratifiedKFold

#時系列分析に必要なライブラリをインポート
import statsmodels.api as sm

#Scikit-learnをインポート
from sklearn.ensemble import RandomForestRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

# データのインポート

In [60]:
#店舗マスタデータのインポート
store_data = pd.read_csv('dataset/store.csv')

In [61]:
#プロモーションコストデータのインポート
promo_cost_data = pd.read_csv('dataset/promo_cost.csv')

In [62]:
#学習データのインポート
train_data = pd.read_csv('dataset/train.csv')

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [63]:
#実際のデータのインポート
test_data = pd.read_csv('dataset/test.csv')

# 学習データの前処理

In [74]:
#テーブルの結合
train_data_m = pd.merge(train_data, store_data, on='Store', how='left')

bk_items = ['Date','StoreType','Assortment']
bk_cols = [col for col in train_data_m.columns if col in bk_items]
train_bk = train_data_m[bk_cols]
train_data_m.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [78]:
#ターゲット変数と説明変数の設定
target_col = 'Sales'
include_cols = ['Id','Store','Promo','Open','Date','DayOfWeek','Year','Month','StoreType','Assortment','Customers']
feature_cols = [col for col in train_data_m.columns if col in include_cols]

#説明変数のデータ
train_data_m['Year'] = train_data_m['Date'].str[0:4]
train_data_m['Month'] = train_data_m['Date'].str[5:7]
train_data_m = train_data_m[train_data_m['Month']<="10"]
train_data_m = train_data_m[train_data_m['Date'].str[5:10]>="03-09"]

y_train = train_data_m[target_col]
x_train = train_data_m[feature_cols]
x_train['Open'] = x_train['Open'].replace(np.NaN,0)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [79]:
# カテゴリ変数のダミー変数作成
colnames_categorical = ['StoreType','Assortment','Month']

x_train_dummy = pd.get_dummies(x_train[colnames_categorical], drop_first=True)

# ダミー変数の結合
x_train = pd.merge(x_train, x_train_dummy, left_index=True, right_index=True)

In [80]:
# 使わない、重複している列の削除
exclude_cols = ['Id','Promo2SinceWeek','Promo2SinceYear','PromoInterval','StoreType','Assortment','Date']
feature_cols = [col for col in x_train.columns if col not in exclude_cols]
x_train = x_train[feature_cols]

In [81]:
train_bk.head()

,Date,StoreType,Assortment
0,2015-07-31,c,a
1,2015-07-31,a,a
2,2015-07-31,a,a
3,2015-07-31,c,c
4,2015-07-31,a,a


# テストデータの前処理

In [82]:
#テーブルの結合
test_data_m = pd.merge(test_data, store_data, on='Store', how='left')
    
bk_cols = [col for col in test_data_m.columns if col in bk_items]
test_bk = test_data_m[bk_cols]

In [83]:
#説明変数のデータ
include_cols = ['Id','Store','Promo','Open','Date','DayOfWeek','StoreType','Assortment','Customers']
feature_cols = [col for col in test_data_m.columns if col in include_cols]

#説明変数のデータ
x_test = test_data_m[feature_cols]
x_test['Open'] = x_test['Open'].replace(np.NaN,0)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [84]:
#説明変数の置換
x_test['Year'] = x_test['Date'].str[0:4]
x_test['Month'] = x_test['Date'].str[5:7]

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [85]:
# カテゴリ変数のダミー変数作成
colnames_categorical = ['StoreType','Assortment','Month']
x_test_dummy = pd.get_dummies(x_test[colnames_categorical], drop_first=True)

# ダミー変数の結合
x_test = pd.merge(x_test, x_test_dummy, left_index=True, right_index=True)

In [86]:
# 使わない、重複している列の削除
exclude_cols = ['Id','Promo2SinceWeek','Promo2SinceYear','PromoInterval','StoreType','Assortment','Date']
feature_cols = [col for col in x_test.columns if col not in exclude_cols]

x_test = x_test[feature_cols]

In [87]:
test_bk.head()

,Date,StoreType,Assortment
0,2015-09-17,c,a
1,2015-09-17,a,a
2,2015-09-17,a,c
3,2015-09-17,a,a
4,2015-09-17,a,c


# 特徴量の選択

In [88]:
#学習データの列数をテストデータとそろえる
x_train = x_train[x_test.columns]
x_train_origin_col = x_train.columns
x_train.head()

,Store,DayOfWeek,Open,Promo,Year,Month,StoreType_b,StoreType_c,StoreType_d,Assortment_b,Assortment_c,Month_09
0,1,5,1,1,2015,07,0,1,0,0,0,0
1,2,5,1,1,2015,07,0,0,0,0,0,0
2,3,5,1,1,2015,07,0,0,0,0,0,0
3,4,5,1,1,2015,07,0,1,0,0,1,0
4,5,5,1,1,2015,07,0,0,0,0,0,0


# 学習処理

In [ ]:
def varImpPlot(clf, feature_names, topN = 10, verbose = True):
    feature_names = np.array(feature_names)
    feature_importance = clf.feature_importances_
    feature_importance = 100.0 * (feature_importance / feature_importance.max())
    sorted_idx = np.argsort(feature_importance)
    
    topN_feature = feature_importance[sorted_idx][len(feature_importance) - topN:]
    topN_feature_name = feature_names[sorted_idx][len(feature_importance) - topN:]
    pos = np.arange(topN) + .5
    plt.barh(pos, topN_feature, align='center')
    plt.yticks(pos, topN_feature_name)
    plt.xlabel('Relative Importance')
    plt.title('Variable Importance')
    plt.show()
    
    rev_sorted_idx = np.argsort(feature_importance)[::-1]
    if verbose:
        for i, v in zip(feature_names[rev_sorted_idx], feature_importance[rev_sorted_idx]):
            print(i, v)

In [89]:
#ターゲット変数と説明変数の設定
feature_cols = [col for col in x_train.columns if col in x_train_origin_col]

#説明変数のデータ
x_train_sel = x_train[feature_cols]
x_train_sel.head()

,Store,DayOfWeek,Open,Promo,Year,Month,StoreType_b,StoreType_c,StoreType_d,Assortment_b,Assortment_c,Month_09
0,1,5,1,1,2015,07,0,1,0,0,0,0
1,2,5,1,1,2015,07,0,0,0,0,0,0
2,3,5,1,1,2015,07,0,0,0,0,0,0
3,4,5,1,1,2015,07,0,1,0,0,1,0
4,5,5,1,1,2015,07,0,0,0,0,0,0


In [90]:
#学習データを、学習用とパラメータチューニング用に分割
X_train1, X_train2, Y_train1, Y_train2 = \
    train_test_split(x_train_sel, y_train, test_size=0.3, random_state=1234)

In [91]:
#rf = RandomForestRegressor(n_estimators = 100, max_depth = 50, max_leaf_nodes = 2000)
rf = RandomForestRegressor()
rf.fit(X_train1, Y_train1)
rf.score(X_train1, Y_train1)

0.97180424545101762

#グリッドサーチ
parameters = {
    'n_estimators':[200,300,500],
    'max_depth':[5,10,50]
}

rf = GridSearchCV(RandomForestRegressor(), parameters)
rf.fit(X_train1, Y_train1)

print(rf.best_estimator_)

In [92]:
#モデルに伴う予測値
Y_pred1 = rf.predict(X_train1)
Y_pred2 = rf.predict(X_train2)

In [93]:
# MSEの計算
from sklearn.metrics import mean_squared_error
print('MSE train : %.3f, test : %.3f' % (mean_squared_error(Y_train1, Y_pred1), mean_squared_error(Y_train2, Y_pred2)) )

# R^2の計算
from sklearn.metrics import r2_score
print('R^2 train : %.3f, test : %.3f' % (r2_score(Y_train1, Y_pred1), r2_score(Y_train2, Y_pred2)) )

MSE train : 402848.199, test : 1160177.099
R^2 train : 0.972, test : 0.919


# 予測処理

In [94]:
feature_cols = [col for col in x_test.columns if col in x_train_sel.columns]

#説明変数のデータ
x_test_sel = x_test[feature_cols]
x_test_sel.head()

,Store,DayOfWeek,Open,Promo,Year,Month,StoreType_b,StoreType_c,StoreType_d,Assortment_b,Assortment_c,Month_09
0,1,4,1.0,1,2015,09,0,1,0,0,0,1
1,3,4,1.0,1,2015,09,0,0,0,0,0,1
2,7,4,1.0,1,2015,09,0,0,0,0,1,1
3,8,4,1.0,1,2015,09,0,0,0,0,0,1
4,9,4,1.0,1,2015,09,0,0,0,0,1,1


In [95]:
#予測の実施
Y_pred = rf.predict(x_test_sel)

In [96]:
test_data['Sales'] = Y_pred

In [97]:
test_data.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Sales
0,1,1,4,2015-09-17,1.0,1,0,0,4108.650000
1,2,3,4,2015-09-17,1.0,1,0,0,7535.200000
2,3,7,4,2015-09-17,1.0,1,0,0,11102.550000
3,4,8,4,2015-09-17,1.0,1,0,0,6576.000000
4,5,9,4,2015-09-17,1.0,1,0,0,7901.766667


In [98]:
#データの書き出し
outfile = DataFrame({'Id': test_data['Id'], 'Sales': test_data['Sales']})
outfile.to_csv("Result.csv", index=False)

In [99]:
outfile.head()

,Id,Sales
0,1,4108.650000
1,2,7535.200000
2,3,11102.550000
3,4,6576.000000
4,5,7901.766667


# プロモーションの効果検証

In [100]:
#説明変数のデータ
x_test_sel2 = x_test[feature_cols]
x_test_sel2['Promo'] = 1
x_test_sel2.head()

,Store,DayOfWeek,Open,Promo,Year,Month,StoreType_b,StoreType_c,StoreType_d,Assortment_b,Assortment_c,Month_09
0,1,4,1.0,1,2015,09,0,1,0,0,0,1
1,3,4,1.0,1,2015,09,0,0,0,0,0,1
2,7,4,1.0,1,2015,09,0,0,0,0,1,1
3,8,4,1.0,1,2015,09,0,0,0,0,0,1
4,9,4,1.0,1,2015,09,0,0,0,0,1,1


In [101]:
#予測の実施
Y_pred2 = rf.predict(x_test_sel2)

In [102]:
test_data['Sales_Promo'] = Y_pred2

In [103]:
test_data.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Sales,Sales_Promo
0,1,1,4,2015-09-17,1.0,1,0,0,4108.650000,4108.650000
1,2,3,4,2015-09-17,1.0,1,0,0,7535.200000,7535.200000
2,3,7,4,2015-09-17,1.0,1,0,0,11102.550000,11102.550000
3,4,8,4,2015-09-17,1.0,1,0,0,6576.000000,6576.000000
4,5,9,4,2015-09-17,1.0,1,0,0,7901.766667,7901.766667


In [104]:
#データの書き出し
outfile2 = DataFrame({'Id': test_data['Id'], 'Store': test_data['Store'], 'Sales': test_data['Sales'], 'Sales_Promo': test_data['Sales_Promo']})
outfile2.to_csv("Result2.csv", index=False)

In [105]:
#テストデータの集計
test_sum = test_data.groupby('Store', as_index=False)['Sales','Sales_Promo'].sum()

In [106]:
test_sum.head()

,Store,Sales,Sales_Promo
0,1,170566.930476,191643.991667
1,3,266563.406710,325355.930000
2,7,418798.570238,511540.266667
3,8,240373.943333,277287.991667
4,9,312088.022857,367701.316667


In [107]:
promo_cost_data.head()

,Store,Cost
0,1,41192
1,2,68495
2,3,65185
3,4,51205
4,5,54793


In [108]:
#テーブルの結合
test_sum2 = pd.merge(test_sum, promo_cost_data, on='Store', how='left')
test_sum2['ROI'] = (test_sum2['Sales_Promo']-test_sum2['Sales']-test_sum2['Cost'])/test_sum2['Cost']
test_sum2.head()

,Store,Sales,Sales_Promo,Cost,ROI
0,1,170566.930476,191643.991667,41192,-0.488321
1,3,266563.406710,325355.930000,65185,-0.098067
2,7,418798.570238,511540.266667,68277,0.358315
3,8,240373.943333,277287.991667,62375,-0.408192
4,9,312088.022857,367701.316667,49110,0.132423


In [109]:
#データの書き出し
outfile3 = DataFrame({'Store': test_sum2['Store'], 'Sales': test_sum2['Sales'], 'Sales_Promo': test_sum2['Sales_Promo'], 'Cost': test_sum2['Cost'], 'ROI': test_sum2['ROI']})
outfile3.to_csv("Result3.csv", index=False)

# 表示用結合

In [110]:
train_data['Type'] = 'Actual'
test_data['Type'] = 'Predict'
test_data['Type2'] = 'Predict_promo'

all_data = DataFrame({'Type': train_data['Type'], 'Store': train_data['Store'], 'Date': train_bk['Date'], 'DayOfWeek': train_data['DayOfWeek'], 'StoreType': train_bk['StoreType'], 'Assortment': train_bk['Assortment'], 'Open': train_data['Open'], 'Promo': train_data['Promo'], 'Sales': train_data['Sales']})
all_data_test = DataFrame({'Type': test_data['Type'], 'Store': test_data['Store'], 'Date': test_bk['Date'], 'DayOfWeek': test_data['DayOfWeek'], 'StoreType': test_bk['StoreType'], 'Assortment': test_bk['Assortment'], 'Open': test_data['Open'], 'Promo': test_data['Promo'], 'Sales': test_data['Sales']})
all_data_test2 = DataFrame({'Type': test_data['Type2'], 'Store': test_data['Store'], 'Date': test_bk['Date'], 'DayOfWeek': test_data['DayOfWeek'], 'StoreType': test_bk['StoreType'], 'Assortment': test_bk['Assortment'], 'Open': test_data['Open'], 'Promo': test_data['Promo'], 'Sales': test_data['Sales_Promo']})

all_data = all_data.append(all_data_test)
all_data = all_data.append(all_data_test2)
all_data.to_csv("TableauData.csv", index=False)

all_data.head()

,Assortment,Date,DayOfWeek,Open,Promo,Sales,Store,StoreType,Type
0,a,2015-07-31,5,1.0,1,5263.0,1,c,Actual
1,a,2015-07-31,5,1.0,1,6064.0,2,a,Actual
2,a,2015-07-31,5,1.0,1,8314.0,3,a,Actual
3,c,2015-07-31,5,1.0,1,13995.0,4,c,Actual
4,a,2015-07-31,5,1.0,1,4822.0,5,a,Actual
